In [ ]:
import pandas as pd
data = pd.read_csv("abstracts.csv")
data = data.drop('Unnamed: 0', axis=1)
data

## Get ChatGPT Triples

In [ ]:
import openai

def chatGPTQuery(label,abstract):
    
    openai.api_key = "KEY"

    chatquery = ("""output just rdf like triples for the entity '%s' from the text:
                 %s""" %(label, abstract))
    
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
            {"role": "system", "content": "You are a chatbot"},
            {"role": "user", "content": chatquery},
        ]
        )

    result = ''
    for choice in response.choices:
        result += choice.message.content

    return result

triples = []
for s,p,o,label,dbosource,abstract in data.values:
    if len(abstract) > 0:
        triples.append(chatGPTQuery(label,abstract))
    else:
        triples.append('')

data['Triples'] = triples
triples[0]

## Extract Triples from String

In [ ]:
# Need to see strings to see how to export

## Export triples

In [ ]:
# Export triples into .ttl file 
#  # add to ChatGPT version of graphDB